In [1]:
#In the prototype, we focus on "text recognition" only 
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold=KFold(n_splits=10)

In [2]:
# reading in the CSV files as Pandas dataframes
df_train = pd.read_csv('vizwiz_skill_typ_train.csv', skipinitialspace = True, engine='python')
df_val = pd.read_csv('vizwiz_skill_typ_val.csv', skipinitialspace = True, engine='python')
# The structure of VizWiz file
df_train.head(3)

,QID,IMG,QSN,ANS_TYP,TXT,OBJ,COL,CNT,OTH,ANS1,ANS2,ANS3,ANS4,ANS5,ANS6,ANS7,ANS8,ANS9,ANS10
0,VizWiz_train_000000000000.jpg,VizWiz_train_000000000000.jpg,What's the name of this product?,other,1,1,0,0,0,basil leaves,basil leaves,basil,basil,basil leaves,basil leaves,basil leaves,basil leaves,basil leaves,basil
1,VizWiz_train_000000000001.jpg,VizWiz_train_000000000001.jpg,Can you tell me what is in this can please?,other,1,1,0,0,0,soda,coca cola,coca cola,unsuitable,unsuitable,coke 0,coca cola 0,coke 0,coca cola,coke
2,VizWiz_train_000000000002.jpg,VizWiz_train_000000000002.jpg,Is this enchilada sauce or is this tomatoes? ...,other,1,1,0,0,0,these tomatoes not enchilada sauce,tomatoes,tomatoes,tomatoes,tomatoes,crushed tomatoes,crushed tomatoes,tomatoes,tomatoes,tomatoes


In [3]:
#create X and Ys
ques_train=df_train["QSN"].values
txt_train=df_train["TXT"].values
obj_train=df_train["OBJ"].values
col_train=df_train["COL"].values
cnt_train=df_train["CNT"].values

ques_val=df_val["QSN"].values
txt_val=df_val["TXT"].values
obj_val=df_val["OBJ"].values
col_val=df_val["COL"].values
cnt_val=df_val["CNT"].values

In [4]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
#do not move stopwords as they are possibly important
#count=CountVectorizer(stop_words='english')
count=CountVectorizer()
bag_train=count.fit_transform(ques_train)#create vocabulary
bag_val=count.transform(ques_val)
print(count.vocabulary_)

{'what': 2697, 'the': 2448, 'name': 1587, 'of': 1655, 'this': 2470, 'product': 1880, 'can': 397, 'you': 2787, 'tell': 2428, 'me': 1489, 'is': 1274, 'in': 1233, 'please': 1815, 'enchilada': 857, 'sauce': 2035, 'or': 1682, 'tomatoes': 2514, 'thank': 2444, 'captcha': 419, 'on': 1668, 'screenshot': 2061, 'item': 1283, 'bottle': 316, 'color': 569, 'do': 776, 'these': 2456, 'look': 1429, 'surface': 2366, 'clean': 521, 'kind': 1327, 'drink': 812, 'skirt': 2172, 'how': 1203, 'much': 1567, 'money': 1550, 'table': 2385, 'ask': 148, 'about': 31, 'anything': 113, 'any': 109, 'information': 1244, 'without': 2735, 'taking': 2396, 'pictures': 1785, 'game': 1032, 'movie': 1565, 'see': 2078, 'piece': 1787, 'mail': 1451, 'and': 98, 'who': 2714, 'for': 980, 'picture': 1783, 'says': 2042, 'does': 781, 'box': 322, 'say': 2040, 'furniture': 1025, 'that': 2446, 'nice': 1614, 'chairs': 468, 'medication': 1499, 'just': 1310, 'like': 1400, 'to': 2506, 'know': 1343, 'if': 1226, 'there': 2453, 'label': 1348, 'dos

In [5]:
quesBOWFeatures_train=bag_train.toarray()
quesBOWFeatures_val=bag_val.toarray()

In [6]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf=TfidfTransformer(use_idf=True,
                      norm= 'l2',
                      smooth_idf=True)
np.set_printoptions(precision=2)
tfidfResultTrain=tfidf.fit_transform(bag_train)
tfidfResultVal=tfidf.transform(bag_val)

In [7]:
quesTFIDFFeatures_train=tfidfResultTrain.toarray()
quesTFIDFFeatures_val=tfidfResultVal.toarray()
print(quesTFIDFFeatures_train.shape)
print(quesTFIDFFeatures_val.shape)

(14269, 2793)
(2251, 2793)


In [9]:
#Try several models to predict
#X is question features, Y is whether it need text recognition
NaiveBayes=GaussianNB()
KNN=KNeighborsClassifier()
SVM=SVC()
Tree=DecisionTreeClassifier()

#fold_accuracies=cross_val_score(NaiveBayes,quesTFIDFFeatures_train,txt_train,cv=kfold)
NaiveBayes.fit(quesTFIDFFeatures_train,txt_train)
testScore=NaiveBayes.score(quesTFIDFFeatures_val,txt_val)
print("Guassian Naive Bayes")
#print("Cross-validation Score:\n{}".format(fold_accuracies))
#print("Average cross-validation Score:{:.2f}".format(fold_accuracies.mean()))
print("Test Score:{:.2f}\n".format(testScore))

#fold_accuracies=cross_val_score(SVM,quesTFIDFFeatures_train,txt_train,cv=kfold)
SVM.fit(quesTFIDFFeatures_train,txt_train)
testScore=SVM.score(quesTFIDFFeatures_val,txt_val)
print("SVM")
#print("Cross-validation Score:\n{}".format(fold_accuracies))
#print("Average cross-validation Score:{:.2f}".format(fold_accuracies.mean()))
print("Test Score:{:.2f}".format(testScore))


#fold_accuracies=cross_val_score(Tree,quesTFIDFFeatures_train,txt_train,cv=kfold)
Tree.fit(quesTFIDFFeatures_train,txt_train)
testScore=Tree.score(quesTFIDFFeatures_val,txt_val)
print("Tree")
#print("Cross-validation Score:\n{}".format(fold_accuracies))
#print("Average cross-validation Score:{:.2f}".format(fold_accuracies.mean()))
print("Test Score:{:.2f}".format(testScore))

Guassian Naive Bayes
Test Score:0.51

SVM
Test Score:0.59
Tree
Test Score:0.78


In [10]:
#fold_accuracies=cross_val_score(KNN,quesTFIDFFeatures_train,txt_train,cv=kfold)
KNN.fit(quesTFIDFFeatures_train,txt_train)
testScore=KNN.score(quesTFIDFFeatures_val,txt_val)
print("KNN")
#print("Cross-validation Score:\n{}".format(fold_accuracies))
#print("Average cross-validation Score:{:.2f}".format(fold_accuracies.mean()))
print("Test Score:{:.2f}".format(testScore))

KNN
Test Score:0.76
